<style>div.title-slide {    width: 100%;    display: flex;    flex-direction: row;            /* default value; can be omitted */    flex-wrap: nowrap;              /* default value; can be omitted */    justify-content: space-between;}</style><div class="title-slide">
<span style="float:left;">Licence CC BY-NC-ND</span>
<span>Thierry Parmentelat &amp; Arnaud Legout</span>
<span><img src="media/both-logos-small-alpha.png" style="display:inline" /></span>
</div>


# `DataFrame` en pandas

## Complément - niveau intermédiaire

### Création d'une DataFrame

Une DataFrame est un tableau numpy à deux dimension avec un index pour les lignes et un index pour les colonnes. Il y a de nombreuses manières de construire une DataFrame.

In [4]:
# Regardons la construction d'une DataFrame
import numpy as np
import pandas as pd

# Créons une Serie pour définir des ages
age = pd.Series([30, 20, 50], index=['alice', 'bob', 'julie'])

# et une Serie pour définir des tailles
height = pd.Series([150, 170, 168], index=['alice', 'marc', 'julie'])

# On peut maintenant combiner ces deux Series en DataFrame,
# chaque Series définissant une colonne, une manière de le faire est 
# de définir un dictionnaire qui contient pour clef le nom de la colonne
# et pour valeur la Series correspondante
stat = pd.DataFrame({'age': age, 'height':height})
print(stat)

        age  height
alice  30.0   150.0
bob    20.0     NaN
julie  50.0   168.0
marc    NaN   170.0


On remarque que pandas fait automatiquement l'alignement des index, lorsqu'une valeur n'est pas présente, elle est automatiquement remplacée par `NaN`. Pandas va également broadcaster une valeur unique définissant un colonne sur toutes les lignes. Regardons cela

In [6]:
stat = pd.DataFrame({'age': age, 'height':height, 'city': 'Nice'})
print(stat)

        age  city  height
alice  30.0  Nice   150.0
bob    20.0  Nice     NaN
julie  50.0  Nice   168.0
marc    NaN  Nice   170.0


In [7]:
# On peut maitenant accéder aux indexes des lignes et des colonnes

# l'index des lignes
print(stat.index)

# l'index des colonnes
print(stat.columns)

Index(['alice', 'bob', 'julie', 'marc'], dtype='object')
Index(['age', 'city', 'height'], dtype='object')


Il y a de nombreuses manières d'accéder maintenant aux éléments de la DataFrame, certaines sont bonnes et d'autres à proscrire, commençons par prendre de bonnes habitudes. Comme il s'agit d'une structure à deux dimensions, il faut donner un indice de ligne et de colonne.

In [25]:
# Quel est l'age de alice
a = stat.loc['alice', 'age']
print(f"l'age de alice est : {a}")

# Quel est la moyenne de tous les ages
m = stat.loc[:, 'age'].mean()
print(f"L'age moyen est de {m:.1f} ans")

l'age de alice est : 30.0
L'age moyen est de 33.3 ans


In [26]:
stat.loc[:, 'age'].mean()

33.333333333333336

On peut déjà noter plusieurs choses intéressantes

 - On peut utiliser `.loc[]` et `.iloc` comme pour les Series. Pour les DataFrame c'est encore plus important parce qu'il y a plus de risques d'ambiguïtés (notamment entre les lignes et le colonnes, on y reviendra). 
 - la méthode `mean` calcule la moyenne, ça n'est pas surprenant, mais ignore les `NaN`. C'est en général ce que l'on veut. Si vous vous demandez comment savoir si la méthode que vous utilisez ignore ou pas les `NaN`, le mieux est de regarder l'aide de cette méthode. Il existe pour un certain nombre de méthodes deux versions : une qui ignore les `NaN` et une autre qui les prend en compte&nbsp;; on reviendra dessus.

Une autre manière de construire une DataFrame est de partir d'un array numpy et de spécifier les indexes pour les lignes et les colonnes avec les arguments `index` et `columns`

In [28]:
a = np.random.randint(1, 20, 9).reshape(3,3)
p = pd.DataFrame(a, index=['a', 'b', 'c'], columns=['x', 'y', 'z'])
print(p)

    x  y   z
a   1  9  10
b   4  9  11
c  11  5   9


### Manipulation d'une DataFrame

In [29]:
# contruisons maintenant une DataFrame jouet

# voici une liste de prénoms
names = ['alice', 'bob', 'marc', 'bill', 'sonia']

# créons trois Series qui formeront trois colonnes
age = pd.Series([12, 13, 16, 11, 16], index=names)
height = pd.Series([130, 140, 176, 120, 165], index=names)
sex = pd.Series(list('fmmmf'), index=names)

# créons maintenant la DataFrame
p = pd.DataFrame({'age':age, 'height':height, 'sex':sex})
print(p)

       age  height sex
alice   12     130   f
bob     13     140   m
marc    16     176   m
bill    11     120   m
sonia   16     165   f


In [32]:
# et chargeons le jeux de données sur les pourboires de seaborn
import seaborn as sns
tips = sns.load_dataset('tips')

Pandas offre de nombreuses possibilités d'explorer les données. Attention, dans mes exemples je vais alterner entre le DataFrame `p` et le DataFrame `tips` suivant les besoins de l'explication. 

In [33]:
# afficher les premières lignes 
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [34]:
# et les dernière lignes
tips.tail()

,total_bill,tip,sex,smoker,day,time,size
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2
243,18.78,3.00,Female,No,Thur,Dinner,2


In [35]:
# afficher l'index des lignes
p.index

Index(['alice', 'bob', 'marc', 'bill', 'sonia'], dtype='object')

In [36]:
# et de colonnes
p.columns

Index(['age', 'height', 'sex'], dtype='object')

In [37]:
# transposer 
p.T

,alice,bob,marc,bill,sonia
age,12,13,16,11,16
height,130,140,176,120,165
sex,f,m,m,m,f


In [38]:
# et afficher uniquement les valeurs
p.values

array([[12, 130, 'f'],
       [13, 140, 'm'],
       [16, 176, 'm'],
       [11, 120, 'm'],
       [16, 165, 'f']], dtype=object)

Pour finir, il y a la méthodes `describe` qui permet d'obtenir des premières statistiques sur un DataFrame. `describe` permet de calculer des statistiques sur des type numériques, mais aussi sur des types chaînes de caractères. 

In [42]:
# par défaut describe ne prend en compte que les colonnes numériques
p.describe()

,age,height
count,5.000000,5.000000
mean,13.600000,146.200000
std,2.302173,23.605084
min,11.000000,120.000000
25%,12.000000,130.000000
50%,13.000000,140.000000
75%,16.000000,165.000000
max,16.000000,176.000000


In [43]:
# mais on peut le forcer en prendre en compte toutes les colonnes
p.describe(include='all')

,age,height,sex
count,5.000000,5.000000,5
unique,NaN,NaN,2
top,NaN,NaN,m
freq,NaN,NaN,3
mean,13.600000,146.200000,NaN
std,2.302173,23.605084,NaN
min,11.000000,120.000000,NaN
25%,12.000000,130.000000,NaN
50%,13.000000,140.000000,NaN
75%,16.000000,165.000000,NaN


### Requêtes sur une DataFrame

On peut maintenant commencer à faire des requêtes sur les DataFrames. Les DataFrame supportent la notion de masque que l'on a vue pour les ndarray numpy et pour les Series. 

In [67]:
# p.loc prend soit un label de ligne
print(p.loc['sonia'])

age        16
height    165
sex         f
Name: sonia, dtype: object


In [68]:
# ou alors un label de ligne ET de colonne
print(p.loc['sonia', 'age'])

16


On peut mettre à la place d'une label :

 - une liste de labels
 - un slice sur les labels
 - un masque (c'est-à-dire un tableau de booléens)
 - un callable qui retourne une des trois premières possibilités
 
Noter que l'on peut également utiliser la notation `.iloc[]` avec les mêmes règles, mais elle est moins utile. 

Je recommande de toujours utiliser la notation `.loc[lignes, colonnes]` pour éviter toute ambiguïté. Nous verrons que les notations `.loc[lignes]` ou pire seulement `[label]` sont sources d'erreurs.

Regardons maintenant d'autres exemples plus sophistiqués.

In [61]:
# gardons uniquement les femmes
p.loc[p.loc[:,'sex']=='f',:]

,age,height,sex
alice,12,130,f
sonia,16,165,f


In [62]:
# gardons uniquement les femmes de plus de 12 ans
p.loc[(p.loc[:,'sex']=='f') & (p.loc[:, 'age'] > 12), :]

,age,height,sex
sonia,16,165,f


In [60]:
# quelle est la note moyenne des femmes
note_f = tips.loc[tips.loc[:,'sex']=='Female', 'total_bill'].mean()
print(f"note moyenne des femmes : {note_f:.2f}")

# quelle est la note moyenne des hommes
note_h = tips.loc[tips.loc[:,'sex']=='Male', 'total_bill'].mean()
print(f"note moyenne des hommes : {note_h:.2f}")

# qui laisse le plus grand pourcentage de pourboire : 
# les hommes ou les femmes

pourboire_f  = tips.loc[tips.loc[:,'sex']=='Female', 'tip'].mean()
pourboire_h  = tips.loc[tips.loc[:,'sex']=='Male', 'tip'].mean()

print(f"Les femmes laissent {pourboire_f/note_f:.2%} de pourboire")
print(f"Les hommes laissent {pourboire_h/note_h:.2%} de pourboire")

note moyenne des femmes : 18.06
note moyenne des hommes : 20.74
Les femmes laissent 15.69% de pourboire
Les hommes laissent 14.89% de pourboire


### Erreurs fréquentes et ambiguïtés sur les requêtes

Nous avons vu une manière simple et non ambiguë de faire des requêtes sur les DataFrame, mais nous allons voir qu'il existe d'autres manières qui ont pour seul avantage d'être plus concise, mais sources de nombreuses erreurs. 

**Souvenez-vous, utilisez toujours la notation `.loc[lignes, colonnes]` sinon, soyez sûr de savoir ce qui est réellement calculé**.

In [71]:
# commençons par la notation la plus classique
p['sex'] # prend forcément un label de colonne

alice    f
bob      m
marc     m
bill     m
sonia    f
Name: sex, dtype: object

In [72]:
# mais par contre, si on passe un slice, c'est forcément des lignes
p['alice': 'marc']

,age,height,sex
alice,12,130,f
bob,13,140,m
marc,16,176,m


In [74]:
# on peut même directement accéder à une colonne par son nom
p.age

alice    12
bob      13
marc     16
bill     11
sonia    16
Name: age, dtype: int64

In [99]:
# mais il ne faut jamais le faire parce que si un attribut de même 
# nom existe sur une DataFrame, alors la priorité est donnée à l'attribut
# et non à la colonne

# ajoutons une colonne qui a pour nom une méthode sur les DataFrame
p['mean'] = 1
print(p)

       age  height sex  mean
alice   12     130   f     1
bob     13     140   m     1
marc    16     176   m     1
bill    11     120   m     1
sonia   16     165   f     1


In [100]:
# je peux bien accéder à la colonne sex
p.sex

alice    f
bob      m
marc     m
bill     m
sonia    f
Name: sex, dtype: object

In [101]:
# mais pas à la colonne mean
p.mean

<bound method DataFrame.mean of        age  height sex  mean
alice   12     130   f     1
bob     13     140   m     1
marc    16     176   m     1
bill    11     120   m     1
sonia   16     165   f     1>

In [102]:
# de nouveau, la seule méthode non ambiguë est d'utiliser .loc
p.loc[:,'mean']

alice    1
bob      1
marc     1
bill     1
sonia    1
Name: mean, dtype: int64

In [103]:
# supprimons maintenant la colonne mean en place (sinon drop retourne
# une nouvelle DataFrame)
p.drop(columns='mean', inplace=True)
print(p)

       age  height sex
alice   12     130   f
bob     13     140   m
marc    16     176   m
bill    11     120   m
sonia   16     165   f


In [104]:
p

,age,height,sex
alice,12,130,f
bob,13,140,m
marc,16,176,m
bill,11,120,m
sonia,16,165,f
